In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
# Cargar extensión de Kedro
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [2]:
movies_metadata = catalog.load("movies_metadata")
credits = catalog.load("credits")
ratings = catalog.load("ratings")


[10/27/25 21:28:22] INFO     Loading data from movies_metadata (CSVDataset)...                 ]8;id=855159;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=886406;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/27/25 21:28:23] INFO     Loading data from credits (CSVDataset)...                         ]8;id=265744;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=136582;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/27/25 21:28:26] INFO     Loading data from ratings (CSVDataset)...                         ]8;id=479138;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=507513;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

In [3]:
print(movies_metadata.head())
print(credits.head())
print(ratings.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [4]:
# Seleccionar solo columnas numéricas
numeric_cols = movies_metadata.select_dtypes(include=['float64', 'int64'])

# Calcular correlación con vote_count
correlations = numeric_cols.corr()['vote_count'].sort_values(ascending=False)
print(correlations)

vote_count      1.000000
revenue         0.812022
vote_average    0.123607
runtime         0.113539
Name: vote_count, dtype: float64


In [9]:
# 🌲 Modelo Random Forest 

# Convertir columnas numéricas
numeric_cols = ['revenue', 'budget', 'popularity', 'runtime', 'vote_average', 'vote_count']
for c in numeric_cols:
    movies_metadata[c] = pd.to_numeric(movies_metadata[c], errors='coerce')

# Eliminar filas con nulos en columnas importantes
movies_metadata = movies_metadata.dropna(subset=numeric_cols + ['status'])

# Codificación de la variable categórica 'status'
status_dummies = pd.get_dummies(movies_metadata['status'], drop_first=True)

# Variables predictoras
X = movies_metadata[['revenue', 'budget', 'popularity', 'runtime', 'vote_average']]
X = pd.concat([X, status_dummies], axis=1)

# Variable objetivo
y = movies_metadata['vote_count']

# Dividir en training y testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# 🔧 Entrenamiento del modelo Random Forest
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Métricas del modelo
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("RESULTADOS MODELO RANDOM FOREST")
print("="*50)
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")



RESULTADOS MODELO RANDOM FOREST
MSE: 52539.2697
MAE: 53.1358
RMSE: 229.2145
R²: 0.8047


Conclusión del Modelo Random Forest

El modelo Random Forest logró un R² de 0.8047, explicando aproximadamente el 80% de la variabilidad en el número de votos que recibe una película. El MAE de 53.14 y el RMSE de 229.21 muestran que el modelo tiene un error moderado, pero es bastante preciso para este tipo de datos.

En resumen, el Random Forest es un modelo sólido y confiable para predecir la popularidad de las películas, siendo hasta ahora el mejor modelo obtenido.